In [ ]:
import findspark
import plotly.graph_objects as go
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import SQLContext


findspark.init()
sc = SparkContext("local", "first app")
spark = SparkSession(sc)
sqlContext = SQLContext(sc)

In [ ]:
archivo_entrada = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true', quote='"', delimiter=',').load("entradas/TraficoAereoGt.csv")
rddfilter = archivo_entrada.rdd.map(tuple)

In [ ]:
rddAir= rddfilter.map(lambda word: (word[0].split('/')[1],word[5]))
rddAir.take(10)

In [ ]:
rddCount=rddAir.reduceByKey(lambda a,b: a+b)
print("Conteo total -> %s" % rddCount.collect())

In [ ]:
rddOrder= sc.parallelize(rddCount.sortBy(lambda a: a[1],False).take(3))
print("Top 3 meses con mayor número de pasajeros de salida%s"%rddOrder.collect())

In [ ]:
rddNames = rddOrder.map(lambda x: (x[0]))
rddTotals = rddOrder.map(lambda x: (x[1]))
print(rddNames.collect())
print(rddTotals.collect())

In [ ]:
fig = go.Figure(data=go.Pie(labels=rddNames.collect(),values=rddTotals.collect()))
fig.update_layout(title_text='3 primeros meses con mayor número de pasajeros de salida.',title_font_size=30)
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20)
fig.write_html('R2_A2.html', auto_open=True)